# Run ----->

In [61]:
# Script adapted for MVM data (API GetSamplesBySite)
# Convert the JSON files to csv by flattening th nested data
# Adapted from https://saralgyaan.com/posts/convert-json-to-csv-using-python/
# Authorship Fantine Chabernaud

import csv
import json
import pandas as pd
import os

import datetime


def get_list_of_json_files():
    #Read the folder where the JSON files are stored and returns a list of the files' names
    list_of_files = os.listdir('data_to_convert_input')
    return list_of_files


def create_list_from_json(jsonfile):
    print('Opening file ' , datetime.datetime.now().time())
    with open(jsonfile, 'r', encoding='utf-8') as f:
        data = json.load(f)     
    df = pd.json_normalize(data)
    
    # Columns to drop 
    columns_to_drop = ['CreateDate','DeliveryMethodTypeCode','DeliveryName','EditDate','HasSampleComment','LocationX', 
    'LocationY','MaxDepth','MinDepth','NumSubSamples','IsBiologicalSample','ProductCode','RelativeLocationType',
    'SampleAccredited','SampleAnalysisLab','SampleAnalysisMethod','SampleCommentFirst100','SampleIsSensitive',
    'SampleMediumTypeName','SampleStatus','SamplingMethod','SamplingMethodStandard','SamplingMethodTypeCode',
    'SourceSampleId','StudyName','SubProgramName','SurveyType','UserIsAuthorized','SiteMetadata',
    'SampleMetadataList','TaxonGroupCalculationValues','SampleCoordinate','SampleCoordinateSRS',
    'ObservedPropertyList','SampleLocationWkt','SiteCoordinate','IndexValues','ObservationValues']
    df.drop(columns = columns_to_drop, inplace = True)
    
    #Flatten the nested data
    clean_df = flatten(data,df)
    return clean_df


def flatten(data,df):
    print('Flattening nested data - START ', datetime.datetime.now().time())
    nested_header_list = ['IndexValues','ObservationValues']
    header_list = ['IndexFlagCodes','ProductName','SampleDate','SampleId',
                    'SiteEUId','SiteId','SiteName','SiteType','WaterZone']
    nested_columns_to_drop = ['IndexValues_Id','IndexValues_FomaIndexName',
       'IndexValues_FomaIndexShortName', 'IndexValues_FomaIndexVersionId',
       'IndexValues_IndexVersion', 'IndexValues_InputValuesText',
       'IndexValues_ObservationSetId', 'IndexValues_UnitOfMeasureName',
       'IndexValues_VisibleInUI',
       'ObservationValues_AccessRestriction', 'ObservationValues_AnalysisDate',
       'ObservationValues_AnalysisLab', 'ObservationValues_AnalysisMethod',
       'ObservationValues_DefaultUnitName', 'ObservationValues_DefaultValue',
       'ObservationValues_IsAccredited', 'ObservationValues_IsComplexType',
       'ObservationValues_IsQualitative', 'ObservationValues_ObservationSetId',
       'ObservationValues_ObservedPropertyId',
       'ObservationValues_PropertyAbbrevName',
       'ObservationValues_PropertyAuthorityCode',
       'ObservationValues_PropertyCode', 
       'ObservationValues_QualityCode', 'ObservationValues_ReportedValue',
       'ObservationValues_TaxonSwedishName',
       'ObservationValues_UnitOfMeasureId',
       'ObservationValues_UnitOfMeasureName',
       'ObservationValues_ValueRepresentationTypeName']
    # Are kept only 'ObservationValues_PropertyName' and 'IndexValues_Value'

    # Flatten the nested columns and merge with data 
    for nested_header in nested_header_list:
        my_data = pd.json_normalize(data=data, record_path=nested_header,record_prefix=nested_header+'_', meta=header_list)
        df = df.merge(my_data, how='outer') 
            
    #Create a new df with only vattenkemi rows
    is_vnk = df['ProductName']=='Vattenkemi'
    df_vnk = df[is_vnk]
    print(df_vnk.shape)
        
    #Dropping useless columns
    df_vnk.drop(columns=nested_columns_to_drop, inplace=False, errors='ignore')
    
    # Final result
    print('Flattening nested data - END ', datetime.datetime.now().time())
    return df_vnk   

    
def write_csv():
    list_of_files = get_list_of_json_files()
    #Iteratively open each JSON file to process it and aggregate it to the CSV file in output
    for file in list_of_files:
        print(file)
        if file == list_of_files[0]:
            df = create_list_from_json(f'data_to_convert_input/{file}')  # create the row to be added to csv for each file (json-file)
            print('Printing output')
            df.to_csv('converted_output/json_to_csv_auto_OUTPUT.csv', index=False, mode='w', header=True)
            
        else :
            df = create_list_from_json(f'data_to_convert_input/{file}')  # create the row to be added to csv for each file (json-file)
            print('Printing output')
            df.to_csv('converted_output/json_to_csv_auto_OUTPUT.csv', index=False, mode='a', header=False)
  
    print('Script end ', datetime.datetime.now().time())

        
if __name__=="__main__":
    write_csv()

mvm_dataset_site_101.json
Opening file  17:58:50.813381
Flattening nested data - START  17:58:51.896395
(5263, 40)
Flattening nested data - END  17:58:52.814326
Printing output
mvm_dataset_site_1082.json
Opening file  17:58:52.981131
Flattening nested data - START  17:58:53.153489
(2417, 30)
Flattening nested data - END  17:58:53.291760
Printing output
mvm_dataset_site_109.json
Opening file  17:58:53.323727
Flattening nested data - START  17:58:54.189821
(6550, 40)
Flattening nested data - END  17:58:54.968371
Printing output
mvm_dataset_site_112.json
Opening file  17:58:55.091301
Flattening nested data - START  17:58:55.628151
(3390, 40)
Flattening nested data - END  17:58:56.030574
Printing output
mvm_dataset_site_114.json
Opening file  17:58:56.096536
Flattening nested data - START  17:58:56.571463
(3204, 40)
Flattening nested data - END  17:58:57.029205
Printing output
mvm_dataset_site_116.json
Opening file  17:58:57.087171
Flattening nested data - START  17:58:57.525785
(3151, 40)

(17, 30)
Flattening nested data - END  17:59:23.547124
Printing output
mvm_dataset_site_24489.json
Opening file  17:59:23.550121
Flattening nested data - START  17:59:23.570111
(105, 30)
Flattening nested data - END  17:59:23.605107
Printing output
mvm_dataset_site_24548.json
Opening file  17:59:23.609105
Flattening nested data - START  17:59:23.615101
(17, 30)
Flattening nested data - END  17:59:23.640087
Printing output
mvm_dataset_site_24608.json
Opening file  17:59:23.643086
Flattening nested data - START  17:59:23.652082
(105, 30)
Flattening nested data - END  17:59:23.688044
Printing output
mvm_dataset_site_2469.json
Opening file  17:59:23.693041
Flattening nested data - START  17:59:23.766188
(472, 30)
Flattening nested data - END  17:59:23.820158
Printing output
mvm_dataset_site_24762.json
Opening file  17:59:23.828153
Flattening nested data - START  17:59:23.852124
(167, 30)
Flattening nested data - END  17:59:23.884110
Printing output
mvm_dataset_site_24850.json
Opening file 

(37, 30)
Flattening nested data - END  17:59:26.529004
Printing output
mvm_dataset_site_36006.json
Opening file  17:59:26.552989
Flattening nested data - START  17:59:26.577073
(37, 30)
Flattening nested data - END  17:59:26.605040
Printing output
mvm_dataset_site_3630.json
Opening file  17:59:26.608047
Flattening nested data - START  17:59:26.641612
(0, 40)
Flattening nested data - END  17:59:26.680591
Printing output
mvm_dataset_site_38989.json
Opening file  17:59:26.683702
Flattening nested data - START  17:59:26.696610
(32, 30)
Flattening nested data - END  17:59:26.730591
Printing output
mvm_dataset_site_39019.json
Opening file  17:59:26.735588
Flattening nested data - START  17:59:26.754578
(96, 30)
Flattening nested data - END  17:59:26.794553
Printing output
mvm_dataset_site_39093.json
Opening file  17:59:26.799550
Flattening nested data - START  17:59:26.806567
(31, 30)
Flattening nested data - END  17:59:26.833551
Printing output
mvm_dataset_site_39143.json
Opening file  17:5

(7, 30)
Flattening nested data - END  17:59:28.960474
Printing output
mvm_dataset_site_43914.json
Opening file  17:59:28.964474
Flattening nested data - START  17:59:28.971472
(7, 30)
Flattening nested data - END  17:59:29.004449
Printing output
mvm_dataset_site_43917.json
Opening file  17:59:29.007448
Flattening nested data - START  17:59:29.013444
(7, 30)
Flattening nested data - END  17:59:29.040428
Printing output
mvm_dataset_site_43925.json
Opening file  17:59:29.043785
Flattening nested data - START  17:59:29.049820
(7, 30)
Flattening nested data - END  17:59:29.077820
Printing output
mvm_dataset_site_43926.json
Opening file  17:59:29.080799
Flattening nested data - START  17:59:29.085456
(7, 30)
Flattening nested data - END  17:59:29.110431
Printing output
mvm_dataset_site_43967.json
Opening file  17:59:29.113421
Flattening nested data - START  17:59:29.119432
(7, 30)
Flattening nested data - END  17:59:29.146421
Printing output
mvm_dataset_site_44026.json
Opening file  17:59:29

# <------

# Draft

In [2]:
# Open deeply nested data 
# source https://www.kaggle.com/jboysen/quick-tutorial-flatten-nested-json-in-pandas
from datetime import datetime
import json 
import pandas as pd 

unwanted_columns = ['CreateDate','DeliveryMethodTypeCode','DeliveryName','EditDate','HasSampleComment','LocationX', 
    'LocationY','MaxDepth','MinDepth','NumSubSamples','IsBiologicalSample','ProductCode','RelativeLocationType',
    'SampleAccredited','SampleAnalysisLab','SampleAnalysisMethod','SampleCommentFirst100','SampleIsSensitive',
    'SampleMediumTypeName','SampleStatus','SamplingMethod','SamplingMethodStandard','SamplingMethodTypeCode',
    'SourceSampleId','StudyName','SubProgramName','SurveyType','UserIsAuthorized','SiteMetadata',
                      'SampleMetadataList','TaxonGroupCalculationValues','SampleCoordinate','SampleCoordinateSRS',
                      'ObservedPropertyList','SampleLocationWkt','SiteCoordinate']

def open_file():
    with open('data_to_convert_input/mvm_dataset_site_54.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

def flatten():
    data = open_file()
    print(data)
    
    nested_header_list = ['IndexValues','ObservationValues'] 
    #'SiteMetadata','SampleMetadataList','TaxonGroupCalculationValues'
    
    header_list = ['IndexFlagCodes','ProductName','SampleDate','SampleId',
                    'SiteEUId','SiteId','SiteName','SiteType','WaterZone'] 
    #'CreateDate','DeliveryMethodTypeCode','DeliveryName','EditDate','HasSampleComment','LocationX', 'LocationY',
    #'MaxDepth','MinDepth','NumSubSamples','IsBiologicalSample','ProductCode',,'RelativeLocationType',
    #'SampleAccredited', 'SampleAnalysisLab','SampleAnalysisMethod','SampleCommentFirst100','SampleIsSensitive',
    #'SampleMediumTypeName', 'SampleStatus','SamplingMethod','SamplingMethodStandard','SamplingMethodTypeCode',
    #'SourceSampleId','StudyName','SubProgramName','SurveyType','UserIsAuthorized',
    
    # Create dataframe of the data 
    df = pd.json_normalize(data)
    df.drop(columns = unwanted_columns, axis=1)
    
    # Flatten the nested columns and merge with data 
    for nested_header in nested_header_list:
        my_data = pd.json_normalize(data=data, record_path=nested_header,
                                    record_prefix=nested_header+'_', meta=header_list)
        df = df.merge(my_data, how='outer')


    #Count how many rows with vattenkemi in dataframe
    #https://cmdlinetips.com/2018/02/how-to-subset-pandas-dataframe-based-on-values-of-a-column/
    #print(df['ObservationValues']['ProductName']=='Vattenkemi')
    #serie = df.apply(lambda x: True if x['ObservationValues']['ProductName']=='Vattenkemi' else False , axis=1) 
    #numOfRows = len(serie[serie == True].index)
    #print(numOfRows)
    
    #Create a new df with only vattenkemi rows
    is_vnk = df['ProductName']=='Vattenkemi'
    df_vnk = df[is_vnk]
    print(df_vnk.head(10))
    
    
    # Final result
    df_vnk.to_csv('converted_output/test.csv', index=False, mode='a', header=False)
    
if __name__=="__main__":
    flatten()
    

ValueError: ('Unknown string format:', '/Date(1327580646000+0100)/')

In [17]:
# Check content of csv file
import pandas
df = pandas.read_csv('converted_output/json_to_csv_auto_OUTPUT.csv')
df.head(2)

,CreateDate,DeliveryMethodTypeCode,DeliveryName,EditDate,HasSampleComment,IndexFlagCodes,IndexValues,IsBiologicalSample,LocationX,LocationY,...,SiteMetadata,SiteName,SiteType,SourceSampleId,StudyName,SubProgramName,SurveyType,TaxonGroupCalculationValues,UserIsAuthorized,WaterZone
0,/Date(1327580646000+0100)/,NaN,NaN,/Date(1414686387000+0100)/,False,NaN,[],False,NaN,NaN,...,"[{'Id': 5414062, 'IsInternal': False, 'Label':...",Spjutsjön,Sjö,NaN,Sjöar trendstationer,NMÖ,NaN,[],False,NaN
1,/Date(1327580646000+0100)/,NaN,NaN,/Date(1414686387000+0100)/,False,NaN,[],False,NaN,NaN,...,"[{'Id': 5414062, 'IsInternal': False, 'Label':...",Spjutsjön,Sjö,NaN,Sjöar trendstationer,NMÖ,NaN,[],False,NaN
